In [0]:
%fs ls /FileStore/tables

path name size modificationTime dbfs:/FileStore/tables/Sales_Records_01__data_set.csv Sales_Records_01__data_set.csv 1247263 1694428616000 dbfs:/FileStore/tables/monthly.csv monthly.csv 962162 1694412507000 dbfs:/FileStore/tables/policy.csv policy.csv 1280673 1694406282000 dbfs:/FileStore/tables/resultF1/ resultF1/ 0 0 dbfs:/FileStore/tables/resultF2/ resultF2/ 0 0 dbfs:/FileStore/tables/train_strokes.csv train_strokes.csv 2635787 1694410708000

In [0]:
salesrec01 = spark.read.load('/FileStore/tables/Sales_Records_01__data_set.csv', format = 'csv', inferSchema = True, header = True)

In [0]:
salesrec01.printSchema()
salesrec01.count()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Units Sold: integer (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Unit Cost: double (nullable = true)
 |-- Total Revenue: double (nullable = true)
 |-- Total Cost: double (nullable = true)
 |-- Total Profit: double (nullable = true)

Out[2]: 10000

In [0]:
from pyspark.sql.functions import when, col

In [0]:
revenueDF = salesrec01.withColumn(
    'Revenue Category',
    when((col('Total Revenue') >= 1) & (col('Total Revenue') <= 2500000), 'Low Revenue')
    .when((col('Total Revenue') > 2500000) & (col('Total Revenue') <= 5000000), 'Medium Revenue')
    .otherwise('High Revenue')
)

In [0]:
revenueDF.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+----------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|Revenue Category|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+----------------+
|  Sub-Saharan Africa|                Chad|Office Supplies|       Online|             L|2011-01-27|292494523|2011-02-12|      4484|    651.21|   524.96|   2920025.64|2353920.64|    566105.0|  Medium Revenue|
|              Europe|              Latvia|      Beverages|       Online|             C|2015-12-28|361825549|2016-01-23|      1075|     47.45|    31.79|     51008.75|  

In [0]:
resC = revenueDF.groupBy('Revenue Category').count()

In [0]:
dbutils.help()

This module provides various utilities for users to interact with the rest of Databricks.
 credentials: DatabricksCredentialUtils -> Utilities for interacting with credentials within notebooks data: DataUtils -> Utilities for understanding and interacting with datasets (EXPERIMENTAL) fs: DbfsUtils -> Manipulates the Databricks filesystem (DBFS) from the console jobs: JobsUtils -> Utilities for leveraging jobs features library: LibraryUtils -> Utilities for session isolated libraries meta: MetaUtils -> Methods to hook into the compiler (EXPERIMENTAL) notebook: NotebookUtils -> Utilities for the control flow of a notebook (EXPERIMENTAL) preview: Preview -> Utilities under preview category secrets: SecretUtils -> Provides utilities for leveraging secrets within notebooks widgets: WidgetsUtils -> Methods to create and get bound value of input widgets inside notebooks

In [0]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one